# T81-558: Applications of Deep Neural Networks


# Google CoLab Instructions

The following code ensures that Google CoLab is running the correct version of TensorFlow.
  Running the following code will map your GDrive to ```/content/drive```.

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

Mounted at /content/drive
Note: using Google CoLab
Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


# Part 3.3: Saving and Loading a Keras Neural Network

Complex neural networks will take a long time to fit/train.  It is helpful to be able to save these neural networks so that you can reload them later.  A reloaded neural network will not require retraining.  Keras provides three formats for neural network saving.

* **JSON** - Stores the neural network structure (no weights) in the [JSON file format](https://en.wikipedia.org/wiki/JSON).
* **HDF5** - Stores the complete neural network (with weights) in the [HDF5 file format](https://en.wikipedia.org/wiki/Hierarchical_Data_Format). Do not confuse HDF5 with [HDFS](https://en.wikipedia.org/wiki/Apache_Hadoop).  They are different.  We do not use HDFS in this class.

Usually, you will want to save in HDF5.

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics

save_path = "."

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])

cars = df['name']

# Handle missing value
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())

# Pandas to Numpy
x = df[['cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin']].values
y = df['mpg'].values # regression

# Build the neural network
model = Sequential()
model.add(Dense(25, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(10, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x,y,verbose=2,epochs=100)

# Predict
pred = model.predict(x)

# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y))
print(f"Before save score (RMSE): {score}")

# save neural network structure to JSON (no weights)
model_json = model.to_json()
with open(os.path.join(save_path,"network.json"), "w") as json_file:
    json_file.write(model_json)

# save entire network to HDF5 (save everything, suggested)
model.save(os.path.join(save_path,"network.h5"))

Epoch 1/100
13/13 - 1s - loss: 590272.1875 - 724ms/epoch - 56ms/step
Epoch 2/100
13/13 - 0s - loss: 140512.7969 - 29ms/epoch - 2ms/step
Epoch 3/100
13/13 - 0s - loss: 8374.6436 - 48ms/epoch - 4ms/step
Epoch 4/100
13/13 - 0s - loss: 3797.3333 - 28ms/epoch - 2ms/step
Epoch 5/100
13/13 - 0s - loss: 2389.2222 - 27ms/epoch - 2ms/step
Epoch 6/100
13/13 - 0s - loss: 115.7635 - 34ms/epoch - 3ms/step
Epoch 7/100
13/13 - 0s - loss: 163.0299 - 32ms/epoch - 2ms/step
Epoch 8/100
13/13 - 0s - loss: 57.7442 - 30ms/epoch - 2ms/step
Epoch 9/100
13/13 - 0s - loss: 36.4762 - 31ms/epoch - 2ms/step
Epoch 10/100
13/13 - 0s - loss: 34.1320 - 27ms/epoch - 2ms/step
Epoch 11/100
13/13 - 0s - loss: 31.8362 - 39ms/epoch - 3ms/step
Epoch 12/100
13/13 - 0s - loss: 31.9264 - 48ms/epoch - 4ms/step
Epoch 13/100
13/13 - 0s - loss: 31.7312 - 29ms/epoch - 2ms/step
Epoch 14/100
13/13 - 0s - loss: 31.7660 - 31ms/epoch - 2ms/step
Epoch 15/100
13/13 - 0s - loss: 31.6010 - 29ms/epoch - 2ms/step
Epoch 16/100
13/13 - 0s - loss:

The code below sets up a neural network and reads the data (for predictions), but it does not clear the model directory or fit the neural network. The code loads the weights from the previous fit. Now we reload the network and perform another prediction. The RMSE should match the previous one exactly if we saved and reloaded the neural network correctly.

In [3]:
from tensorflow.keras.models import load_model
model2 = load_model(os.path.join(save_path,"network.h5"))
pred = model2.predict(x)
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y))
print(f"After load score (RMSE): {score}")

13/13 [==============================] - 0s 2ms/step
After load score (RMSE): 4.9457376984188155
